In [ ]:
import ccxt
import pandas as pd
binance = ccxt.binance({
    'apiKey':'heJCdTPU31a1u0zmBcv1f20V1EZnax23O6njULZn2cutz41YtydEFhcOJcOg8dAm',
    'secret':'ESuH2Y4ILi061nQJ6MXGfGD7Qct0OEEl9wf0mI8KMH8SQiCzQx4EubfhJvrHEzZB'
})
binance.load_markets()
new_pairs = [p for p in new_pairs if p in binance.symbols]
len(new_pairs),new_pairs

In [8]:
import pandas as pd
pair100 = ["BTC/USDT","ETH/USDT","BNB/USDT","XRP/USDT","USDC/USDT","SOL/USDT","ADA/USDT","DOGE/USDT","TRX/USDT","TON/USDT","LINK/USDT","MATIC/USDT","DOT/USDT","WBTC/USDT","LTC/USDT","DAI/USDT","SHIB/USDT","BCH/USDT","AVAX/USDT","LEO/USDT","XLM/USDT","TUSD/USDT","ATOM/USDT","OKB/USDT","XMR/USDT","UNI/USDT","ETC/USDT","FIL/USDT","HBAR/USDT","ICP/USDT","CRO/USDT","BUSD/USDT","APT/USDT","LDO/USDT","VET/USDT","NEAR/USDT","INJ/USDT","AAVE/USDT","ARB/USDT","OP/USDT","GRT/USDT","MNT/USDT","MKR/USDT","QNT/USDT","RUNE/USDT","EGLD/USDT","IMX/USDT","ALGO/USDT","STX/USDT","BSV/USDT","NEO/USDT","RNDR/USDT","AXS/USDT","THETA/USDT","SAND/USDT","MANA/USDT","XTZ/USDT","XDC/USDT","EOS/USDT","FTM/USDT","USDD/USDT","SNX/USDT","FDUSD/USDT","KAVA/USDT","TWT/USDT","BGB/USDT","FLOW/USDT","MINA/USDT","CHZ/USDT","XEC/USDT","CFX/USDT","CAKE/USDT","KCS/USDT","APE/USDT","GALA/USDT","IOTA/USDT","CRV/USDT","SUI/USDT","RPL/USDT","AR/USDT","FXS/USDT","ZEC/USDT","PEPE/USDT","XAUt/USDT","PAXG/USDT","KLAY/USDT","USDP/USDT","ROSE/USDT","ETHDYDX/USDT","BTT/USDT","CSPR/USDT","GMX/USDT","WOO/USDT","COMP/USDT","BLUR/USDT","GT/USDT","LUNC/USDT","HT/USDT","NEXO/USDT","RBTC/USDT"]
# results_file = f"../wlogs/backtests/50pairs-trendHedge.csv"
# results = pd.read_csv(results_file)
old_pairs = results['pair'].unique()
old_pairs = [p.replace("USDT","/USDT:") for p in old_pairs]
new_pairs = [p for p in pair100 if p not in old_pairs]



(69,
 ['USDC/USDT',
  'TON/USDT',
  'WBTC/USDT',
  'DAI/USDT',
  'SHIB/USDT',
  'LEO/USDT',
  'TUSD/USDT',
  'OKB/USDT',
  'HBAR/USDT',
  'ICP/USDT',
  'CRO/USDT',
  'BUSD/USDT',
  'APT/USDT',
  'LDO/USDT',
  'VET/USDT',
  'NEAR/USDT',
  'INJ/USDT',
  'AAVE/USDT',
  'ARB/USDT',
  'OP/USDT',
  'GRT/USDT',
  'MNT/USDT',
  'MKR/USDT',
  'RUNE/USDT',
  'EGLD/USDT',
  'IMX/USDT',
  'STX/USDT',
  'BSV/USDT',
  'NEO/USDT',
  'RNDR/USDT',
  'AXS/USDT',
  'MANA/USDT',
  'XDC/USDT',
  'USDD/USDT',
  'SNX/USDT',
  'FDUSD/USDT',
  'TWT/USDT',
  'BGB/USDT',
  'FLOW/USDT',
  'MINA/USDT',
  'XEC/USDT',
  'CFX/USDT',
  'CAKE/USDT',
  'KCS/USDT',
  'APE/USDT',
  'GALA/USDT',
  'CRV/USDT',
  'SUI/USDT',
  'RPL/USDT',
  'AR/USDT',
  'FXS/USDT',
  'PEPE/USDT',
  'XAUt/USDT',
  'PAXG/USDT',
  'KLAY/USDT',
  'USDP/USDT',
  'ROSE/USDT',
  'ETHDYDX/USDT',
  'BTT/USDT',
  'CSPR/USDT',
  'GMX/USDT',
  'WOO/USDT',
  'COMP/USDT',
  'BLUR/USDT',
  'GT/USDT',
  'LUNC/USDT',
  'HT/USDT',
  'NEXO/USDT',
  'RBTC/USDT'

In [62]:
# %load_ext autoreload
# %autoreload 2
import os
from tqdm.notebook import tqdm
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

from hedgeMode import HedgeMode as Strategy
def mean_k(d):
    s = 0
    c = 0
    for k,v in d.items():
        s += k*v
        c += v
    if c==0:
        return None
    return s/c

data_path='/home/admin/freqtrade/user_data/data/binance/futures/'
files = [f for f in os.listdir(data_path) if f.endswith("5Min.csv")]
# files = ['Binance_BNBUSDT_5Min.csv']
sl_range = [0.016,0.018,0.02,0.022,0.025,0.028,0.03]
# sl_range = [0.035,0.04]
market_type = 'trend'
max_fail = 20
init_money=30000
output=f'../wlogs/backtests/{datetime.now().year}-{datetime.now().month}-{datetime.now().day}'


results = []
for f in files:
    df = pd.read_csv(os.path.join(data_path,f),parse_dates=['date'])
    pair = f.split("_")[1]
    start = df['date'].min()
    end = df['date'].max()
    print(f"Pair: {pair}")
    for sl in sl_range:
        vars = {
            'k':1,
            'safety_factor':1.2,
            'kpos':0,
            'start_money':1,
            'increase_factor':1,
            'normal_fail':4,
            'max_fail':max_fail,
            'tp':1+sl,
            'sl':sl,
            'leverage':100,
            'market_type':market_type,
        }
        strategy = Strategy(vars)
        result_df = strategy.back_test(df)
        result_df['balance'] = result_df['long_PNL'].cumsum()+result_df['short_PNL'].cumsum()+init_money
        result_df.to_csv(f"{output}-{pair[:3]}-{strategy.__class__.__name__}-{market_type}-(sl={sl:.3f}).csv",index=False)
        print(f"Number of trades: {len(result_df)}")
        result_df['duration'] = result_df['exit_time']-result_df['entry_time']
#         short_k_dist = result_df['short_k'].value_counts()
#         short_k_dist.at[len(short_k_dist)]=0
#         short_k_dist = short_k_dist.diff(-1).dropna()[1:].astype(int)
#         long_k_dist = result_df['long_k'].value_counts()
#         long_k_dist.at[len(long_k_dist)]=0
#         long_k_dist = short_k_dist.diff(-1).dropna()[1:].astype(int)
        trades1y = result_df[trades['entry_time']>pd.Timestamp(year=2021,month=11,day=1)]
        trades2y = result_df[trades['entry_time']>pd.Timestamp(year=2020,month=11,day=1)]
        results.append({
            'pair':pair,
            'vars':vars,
            'start':{start},
            'end': end,
            'strategy':f"{strategy.__class__.__name__}-{vars['market_type']}",
            'profit':result_df['short_PNL'].sum()+result_df['long_PNL'].sum(),
            'init_balance':init_money,
            'final_balance':result_df.iloc[len(result_df)-1]['balance'],
            'min_balance': result_df['balance'].min(),
            'max_balance': result_df['balance'].max(),
            'average_duration':result_df['duration'].mean(),
            'n_trades':len(result_df),
            'n_fails':strategy.n_fails,
            'max_k': max(result_df['short_k'].max(),result_df['long_k'].max()),
            'max_k_1y': max(trades1y['short_k'].max(),trades1y['long_k'].max()),
            'max_k_2y': max(trades2y['short_k'].max(),trades2y['long_k'].max()),           
            'short_k_distribution': dict(trades['short_k'].value_counts()),
            'long_k_distribution': dict(trades['long_k'].value_counts()),
        })
        print(f"Max k (TP = {sl})= {max(result_df['short_k'].max(),result_df['long_k'].max())}")
    print()
results_file = f"../wlogs/backtests/50pairs-trendHedge.csv"
results = pd.DataFrame(results)
results['tp'] = results['vars'].apply(lambda v: v['sl'])

if not  os.path.exists(results_file):
    results.to_csv(results_file,index=False)
else:
    pre_results = pd.read_csv(results_file)
    results = pd.concat([pre_results,results])
    results.to_csv(results_file,index=False)
results


Pair: BNBUSDT
Number of trades: 5963
Max k (TP = 0.035)= 8-7
Number of trades: 4539
Max k (TP = 0.04)= 8-7



,pair,vars,start,end,strategy,profit,init_balance,final_balance,min_balance,max_balance,average_duration,n_trades,n_fails,short_k_distribution,long_k_distribution,short_k_mean,long_k_mean,short_max_k,long_max_k,tp
0,UNIUSDT,"{'k': 4, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,69779.754404,30000,99779.754404,19718.049444,99796.050404,0 days 01:34:18.963636363,16500,0,"{1: 2084, 2: 1047, 3: 522, 4: 273, 5: 112, 6: ...","{2: 525, 3: 249, 4: 161, 5: 46, 6: 41, 7: 6, 8...",1.978114,2.972275,14,13,0.016
1,UNIUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,48628.493000,30000,78628.493000,23879.051259,78629.893000,0 days 01:58:00.211775307,13316,0,"{1: 1707, 2: 813, 3: 413, 4: 232, 5: 92, 6: 57...","{2: 400, 3: 181, 4: 140, 5: 35, 6: 38, 7: 9, 8...",1.976453,3.011084,13,12,0.018
2,UNIUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,39368.662698,30000,69368.662698,-17535.805240,69370.062698,0 days 02:23:12.851325671,11051,0,"{1: 1427, 2: 666, 3: 385, 4: 148, 5: 106, 6: 3...","{2: 281, 3: 237, 4: 42, 5: 76, 6: 8, 7: 13, 8:...",1.967143,3.049624,11,10,0.020
3,UNIUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,25758.869416,30000,55758.869416,27146.371426,55758.869416,0 days 02:49:40.928000,9375,0,"{1: 1168, 2: 570, 3: 322, 4: 147, 5: 73, 6: 38...","{2: 248, 3: 175, 4: 74, 5: 35, 6: 31, 7: -9, 8...",1.992324,3.015901,10,9,0.022
4,UNIUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,24545.034888,30000,54545.034888,16587.940039,54546.434888,0 days 03:34:07.143812709,7475,0,"{1: 926, 2: 442, 3: 264, 4: 112, 5: 50, 6: 37,...","{2: 178, 3: 152, 4: 62, 5: 13, 6: 29, 7: -3, 8...",1.999461,3.079365,12,11,0.025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,THETAUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2019-04-10 04:00:00')},2023-10-30 09:35:00,HedgeMode-trend,45133.210926,30000,75133.210926,-6335.085407,75134.610926,0 days 03:19:35.868841260,11711,0,"{1: 1241, 2: 797, 3: 302, 4: 216, 5: 78, 6: 67...","{2: 495, 3: 86, 4: 138, 5: 11, 6: 43, 7: 6, 8:...",2.106468,2.875472,11,10,0.025
285,THETAUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2019-04-10 04:00:00')},2023-10-30 09:35:00,HedgeMode-trend,33331.670266,30000,63331.670266,25714.680249,63333.070266,0 days 04:08:58.759936406,9435,0,"{1: 971, 2: 644, 3: 251, 4: 180, 5: 82, 6: 44,...","{2: 393, 3: 71, 4: 98, 5: 38, 6: 29, 7: 8, 8: ...",2.108133,2.883178,12,12,0.028
286,THETAUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2019-04-10 04:00:00')},2023-10-30 09:35:00,HedgeMode-trend,26787.646282,30000,56787.646282,26260.782975,56787.646282,0 days 04:46:15.656614785,8224,0,"{1: 841, 2: 587, 3: 211, 4: 157, 5: 59, 6: 50,...","{2: 376, 3: 54, 4: 98, 5: 9, 6: 46, 7: -4, 8: ...",2.096154,2.758621,10,9,0.030
0,BNBUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{2017-11-06 03:50:00},2023-10-30 08:00:00,HedgeMode-trend,12963.857946,30000,42963.857946,28998.578491,42965.257946,0 days 08:41:04.900218011,5963,0,"{1: 615, 2: 402, 3: 194, 4: 105, 5: 46, 6: 30,...","{2: 208, 3: 89, 4: 59, 5: 16, 6: 29, 7: -7}",2.071378,2.852792,8,7,0.035


In [104]:
files = [f for f in os.listdir(data_path) if f.endswith("5Min.csv")]
pairs = [('1000LUNC/USDT:USDT', '1000LUNC/USDT:USDT'),
    ('1000PEPE/USDT:USDT', '1000PEPE/USDT:USDT'),
    ('1000SHIB/USDT:USDT', '1000SHIB/USDT:USDT'),
    ('1000XEC/USDT:USDT', '1000XEC/USDT:USDT'),
    ('ADA/USDT:USDT', 'ADA/USDT:USDT'),
    ('ALGO/USDT:USDT', 'ALGO/USDT:USDT'),
    ('APT/USDT:USDT', 'APT/USDT:USDT'),
    ('ATOM/USDT:USDT', 'ATOM/USDT:USDT'),
    ('AVAX/USDT:USDT', 'AVAX/USDT:USDT'),
    ('BCH/USDT:USDT', 'BCH/USDT:USDT'),
    ('BNB/USDT:USDT', 'BNB/USDT:USDT'),
    ('BTC/USDT:USDT', 'BTC/USDT:USDT'),
    ('DOGE/USDT:USDT', 'DOGE/USDT:USDT'),
    ('DOT/USDT:USDT', 'DOT/USDT:USDT'),
    ('ETC/USDT:USDT', 'ETC/USDT:USDT'),
    ('ETH/USDT:USDT', 'ETH/USDT:USDT'),
    ('FIL/USDT:USDT', 'FIL/USDT:USDT'),
    ('FTM/USDT:USDT', 'FTM/USDT:USDT'),
    ('GALA/USDT:USDT', 'GALA/USDT:USDT'),
    ('LDO/USDT:USDT', 'LDO/USDT:USDT'),
    ('LINK/USDT:USDT', 'LINK/USDT:USDT'),
    ('LTC/USDT:USDT', 'LTC/USDT:USDT'),
    ('MATIC/USDT:USDT', 'MATIC/USDT:USDT'),
    ('NEAR/USDT:USDT', 'NEAR/USDT:USDT'),
    ('QTUM/USDT:USDT', 'QTUM/USDT:USDT'),
    ('SAND/USDT:USDT', 'SAND/USDT:USDT'),
    ('SOL/USDT:USDT', 'SOL/USDT:USDT'),
    ('SXP/USDT:USDT', 'SXP/USDT:USDT'),
    ('TRX/USDT:USDT', 'TRX/USDT:USDT'),
    ('UNI/USDT:USDT', 'UNI/USDT:USDT'),
    ('XLM/USDT:USDT', 'XLM/USDT:USDT'),
    ('XMR/USDT:USDT', 'XMR/USDT:USDT'),
    ('XRP/USDT:USDT', 'XRP/USDT:USDT'),
    ('XTZ/USDT:USDT', 'XTZ/USDT:USDT'),
    ('BTC/USD:BTC','BTC/USD:BTC'),
    ('BTC/USDT:BTC','BTC/USDT:BTC'),
    ('BTC/USD:BTC-231229','BTC/USD:BTC-231229'),
    ('ETH/BTC:BTC','ETH/BTC:BTC'),
    ('XLM/USD:XLM','XLM/USD:XLM')]
new_pairs = [f.split('_')[1] for f in files]
for np in new_pairs:
    ns = np.replace('USDT','/USDT:USDT')
    if (ns,ns) not in pairs:
        pairs.append((ns,ns))
        print(f"{ns} added to pairs")
pairs

ZRX/USDT:USDT added to pairs
AUDIO/USDT:USDT added to pairs
IOTA/USDT:USDT added to pairs
ZEC/USDT:USDT added to pairs
ZIL/USDT:USDT added to pairs
KAVA/USDT:USDT added to pairs
SUSHI/USDT:USDT added to pairs
CHR/USDT:USDT added to pairs
1INCH/USDT:USDT added to pairs
DASH/USDT:USDT added to pairs
QNT/USDT:USDT added to pairs
EOS/USDT:USDT added to pairs
ENJ/USDT:USDT added to pairs
CHZ/USDT:USDT added to pairs
THETA/USDT:USDT added to pairs


[('1000LUNC/USDT:USDT', '1000LUNC/USDT:USDT'),
 ('1000PEPE/USDT:USDT', '1000PEPE/USDT:USDT'),
 ('1000SHIB/USDT:USDT', '1000SHIB/USDT:USDT'),
 ('1000XEC/USDT:USDT', '1000XEC/USDT:USDT'),
 ('ADA/USDT:USDT', 'ADA/USDT:USDT'),
 ('ALGO/USDT:USDT', 'ALGO/USDT:USDT'),
 ('APT/USDT:USDT', 'APT/USDT:USDT'),
 ('ATOM/USDT:USDT', 'ATOM/USDT:USDT'),
 ('AVAX/USDT:USDT', 'AVAX/USDT:USDT'),
 ('BCH/USDT:USDT', 'BCH/USDT:USDT'),
 ('BNB/USDT:USDT', 'BNB/USDT:USDT'),
 ('BTC/USDT:USDT', 'BTC/USDT:USDT'),
 ('DOGE/USDT:USDT', 'DOGE/USDT:USDT'),
 ('DOT/USDT:USDT', 'DOT/USDT:USDT'),
 ('ETC/USDT:USDT', 'ETC/USDT:USDT'),
 ('ETH/USDT:USDT', 'ETH/USDT:USDT'),
 ('FIL/USDT:USDT', 'FIL/USDT:USDT'),
 ('FTM/USDT:USDT', 'FTM/USDT:USDT'),
 ('GALA/USDT:USDT', 'GALA/USDT:USDT'),
 ('LDO/USDT:USDT', 'LDO/USDT:USDT'),
 ('LINK/USDT:USDT', 'LINK/USDT:USDT'),
 ('LTC/USDT:USDT', 'LTC/USDT:USDT'),
 ('MATIC/USDT:USDT', 'MATIC/USDT:USDT'),
 ('NEAR/USDT:USDT', 'NEAR/USDT:USDT'),
 ('QTUM/USDT:USDT', 'QTUM/USDT:USDT'),
 ('SAND/USDT:USDT

In [92]:
results = results.drop(columns=['short_k_distribution_1y','long_k_distribution_1y'])
# results = results.reset_index()
# results = results.drop(columns=['level_0','index'])
results

,pair,vars,start,end,strategy,profit,init_balance,final_balance,min_balance,max_balance,average_duration,n_trades,n_fails,short_k_mean,long_k_mean,short_max_k,long_max_k,tp,max_k,max_k_1y
0,UNIUSDT,"{'k': 4, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,69779.754404,30000,99779.754404,19718.049444,99796.050404,0 days 01:34:18.963636363,16500,0,1.978114,2.972275,14,13,0.016,14.0,14.0
1,UNIUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,48628.493000,30000,78628.493000,23879.051259,78629.893000,0 days 01:58:00.211775307,13316,0,1.976453,3.011084,13,12,0.018,13.0,13.0
2,UNIUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,39368.662698,30000,69368.662698,-17535.805240,69370.062698,0 days 02:23:12.851325671,11051,0,1.967143,3.049624,11,10,0.020,14.0,NaN
3,UNIUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,25758.869416,30000,55758.869416,27146.371426,55758.869416,0 days 02:49:40.928000,9375,0,1.992324,3.015901,10,9,0.022,NaN,NaN
4,UNIUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-09-17 03:00:00')},2023-10-30 08:55:00,HedgeMode-trend,24545.034888,30000,54545.034888,16587.940039,54546.434888,0 days 03:34:07.143812709,7475,0,1.999461,3.079365,12,11,0.025,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,THETAUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2019-04-10 04:00:00')},2023-10-30 09:35:00,HedgeMode-trend,45133.210926,30000,75133.210926,-6335.085407,75134.610926,0 days 03:19:35.868841260,11711,0,2.106468,2.875472,11,10,0.025,NaN,NaN
285,THETAUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2019-04-10 04:00:00')},2023-10-30 09:35:00,HedgeMode-trend,33331.670266,30000,63331.670266,25714.680249,63333.070266,0 days 04:08:58.759936406,9435,0,2.108133,2.883178,12,12,0.028,NaN,NaN
286,THETAUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2019-04-10 04:00:00')},2023-10-30 09:35:00,HedgeMode-trend,26787.646282,30000,56787.646282,26260.782975,56787.646282,0 days 04:46:15.656614785,8224,0,2.096154,2.758621,10,9,0.030,NaN,NaN
287,BNBUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{2017-11-06 03:50:00},2023-10-30 08:00:00,HedgeMode-trend,12963.857946,30000,42963.857946,28998.578491,42965.257946,0 days 08:41:04.900218011,5963,0,2.071378,2.852792,8,7,0.035,14.0,14.0


In [94]:
for i,r in results.iterrows():
    filename = f'../wlogs/backtests/2023-11-5-{r["pair"][:3]}-HedgeMode-trend-(sl={r["tp"]:.3f}).csv'
    trades = pd.read_csv(filename,parse_dates=['entry_time'])
    results.at[i,'max_k'] = max(trades['short_k'].max(),trades['long_k'].max())
#     dict(trades['short_k'].value_counts())
#     print
#     results.at[i,'short_k_distribution'] = dict(trades['short_k'].value_counts())
#     results.at[i,'long_k_distribution'] = dict(trades['long_k'].value_counts())
    trades = trades[trades['entry_time']>pd.Timestamp(year=2021,month=11,day=1)]
    results.at[i,'max_k_1y'] = max(trades['short_k'].max(),trades['long_k'].max())
#     results.at[i,'short_k_distribution_1y'] = dict(trades['short_k'].value_counts())    
#     results.at[i,'long_k_distribution_1y'] = dict(trades['long_k'].value_counts())

# results['short_k_mean'] = results['short_k_distribution'].apply(mean_k)
# results['long_k_mean'] = results['long_k_distribution'].apply(mean_k)
# results['short_max_k'] = results['short_k_distribution'].apply(lambda d: max(d.keys()))
# results['long_max_k'] = results['long_k_distribution'].apply(lambda d: max(d.keys()))
# results['k_mean'] = (results['long_k_mean']+results['short_k_mean'])/2

results.sort_values(by='max_k_1y')[['pair','tp','max_k','max_k_1y']]

,pair,vars,start,end,strategy,profit,init_balance,final_balance,min_balance,max_balance,average_duration,n_trades,n_fails,short_k_mean,long_k_mean,short_max_k,long_max_k,tp,max_k,max_k_1y
178,BNBUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2017-11-06 03:50:00')},2023-10-30 08:00:00,HedgeMode-trend,4.939032e+04,30000,7.939032e+04,8.692384e+03,7.939600e+04,0 days 03:43:03.157894736,13775,0,2.051568,2.998840,13,12,0.022,13.0,7.0
90,BTCUSDT,"{'k': 4, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2017-08-17 04:00:00')},2023-10-30 07:45:00,HedgeMode-trend,3.482429e+04,30000,6.482429e+04,-9.337836e+04,6.484058e+04,0 days 12:11:35.611852522,4421,0,2.106758,2.992832,14,14,0.030,15.0,8.0
287,BNBUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{2017-11-06 03:50:00},2023-10-30 08:00:00,HedgeMode-trend,1.296386e+04,30000,4.296386e+04,2.899858e+04,4.296526e+04,0 days 08:41:04.900218011,5963,0,2.071378,2.852792,8,7,0.035,9.0,8.0
155,TRXUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2018-06-11 11:30:00')},2023-10-30 08:25:00,HedgeMode-trend,3.643592e+04,30000,6.643592e+04,2.981066e+04,6.643592e+04,0 days 03:18:42.991071428,13888,0,1.998248,2.785714,12,11,0.018,12.0,8.0
160,TRXUSDT,"{'k': 1, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2018-06-11 11:30:00')},2023-10-30 08:25:00,HedgeMode-trend,1.372102e+04,30000,4.372102e+04,1.910780e+04,4.372102e+04,0 days 08:28:25.549510337,5514,0,2.006002,3.019553,12,10,0.030,12.0,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
253,CHZUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2019-09-06 04:00:00')},2023-10-31 07:20:00,HedgeMode-trend,2.015730e+05,30000,2.315730e+05,-5.239392e+05,2.315787e+05,0 days 01:34:50.310627201,21859,0,2.071142,3.046529,14,13,0.018,17.0,17.0
35,AUDIOUSDT,"{'k': 2, 'safety_factor': 1.2, 'kpos': 1, 'sta...",{Timestamp('2020-10-23 19:00:00')},2023-10-31 08:00:00,HedgeMode-trend,3.665408e+05,30000,3.965408e+05,-1.207968e+06,3.965422e+05,0 days 01:02:32.501701837,23504,0,1.965351,3.014403,16,14,0.016,18.0,18.0
162,1INCHUSDT,"{'k': 3, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2020-12-25 05:00:00')},2023-10-31 07:40:00,HedgeMode-trend,7.094404e+05,30000,7.394404e+05,-2.782197e+06,7.394461e+05,0 days 01:43:17.858486470,13822,0,2.088190,3.136307,17,15,0.018,19.0,19.0
24,XLMUSDT,"{'k': 6, 'safety_factor': 1.2, 'kpos': 0, 'sta...",{Timestamp('2018-05-31 09:30:00')},2023-10-30 09:00:00,HedgeMode-trend,6.240044e+05,30000,6.540044e+05,-2.848348e+06,6.541005e+05,0 days 04:01:19.250670589,11557,0,1.999656,3.050360,18,16,0.022,19.0,19.0


In [95]:
results.sort_values(by='max_k_1y')[['pair','tp','max_k','max_k_1y']]

,pair,tp,max_k,max_k_1y
178,BNBUSDT,0.022,13.0,7.0
90,BTCUSDT,0.030,15.0,8.0
287,BNBUSDT,0.035,9.0,8.0
155,TRXUSDT,0.018,12.0,8.0
160,TRXUSDT,0.030,12.0,8.0
...,...,...,...,...
253,CHZUSDT,0.018,17.0,17.0
35,AUDIOUSDT,0.016,18.0,18.0
162,1INCHUSDT,0.018,19.0,19.0
24,XLMUSDT,0.022,19.0,19.0


In [ ]:
resu

In [97]:
for i,r in results.iterrows():
    filename = f'../wlogs/backtests/2023-11-5-{r["pair"][:3]}-HedgeMode-trend-(sl={r["tp"]:.3f}).csv'
    trades = pd.read_csv(filename,parse_dates=['entry_time'])
    trades = trades[trades['entry_time']>pd.Timestamp(year=2020,month=11,day=1)]
    results.at[i,'max_k_2y'] = max(trades['short_k'].max(),trades['long_k'].max())
results.sort_values(by='max_k_2y')[results['tp']==0.02][['pair','tp','max_k','max_k_1y','max_k_2y']]


/tmp/ipykernel_764323/1470700745.py:6: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  results.sort_values(by='max_k_2y')[results['tp']==0.02][['pair','tp','max_k','max_k_1y','max_k_2y']]


,pair,tp,max_k,max_k_1y,max_k_2y
86,BTCUSDT,0.02,10.0,9.0,10.0
191,XMRUSDT,0.02,11.0,10.0,10.0
156,TRXUSDT,0.02,12.0,11.0,11.0
142,ETHUSDT,0.02,11.0,11.0,11.0
121,LINKUSDT,0.02,14.0,11.0,11.0
114,KAVAUSDT,0.02,12.0,11.0,11.0
135,CHRUSDT,0.02,11.0,11.0,11.0
233,ADAUSDT,0.02,14.0,11.0,11.0
212,SXPUSDT,0.02,11.0,9.0,11.0
198,EOSUSDT,0.02,11.0,10.0,11.0


In [100]:
results.sort_values(by='max_k_2y')[['pair','tp','max_k','max_k_1y','max_k_2y','final_balance','min_balance','max_balance','n_trades']]

,pair,tp,max_k,max_k_1y,max_k_2y,final_balance,min_balance,max_balance,n_trades
288,BNBUSDT,0.040,8.0,8.0,8.0,3.958995e+04,2.995499e+04,3.959135e+04,4539
160,TRXUSDT,0.030,12.0,8.0,8.0,4.372102e+04,1.910780e+04,4.372102e+04,5514
159,TRXUSDT,0.028,11.0,9.0,9.0,4.470691e+04,2.560950e+04,4.470691e+04,6249
174,DASHUSDT,0.030,11.0,9.0,9.0,4.478154e+04,2.834194e+04,4.478154e+04,5448
180,BNBUSDT,0.028,10.0,9.0,9.0,5.235630e+04,2.881433e+04,5.245237e+04,8965
...,...,...,...,...,...,...,...,...,...
162,1INCHUSDT,0.018,19.0,19.0,19.0,7.394404e+05,-2.782197e+06,7.394461e+05,13822
24,XLMUSDT,0.022,19.0,19.0,19.0,6.540044e+05,-2.848348e+06,6.541005e+05,11557
77,ZILUSDT,0.016,19.0,12.0,19.0,6.983870e+05,-2.836226e+06,6.983884e+05,26278
79,ZILUSDT,0.020,20.0,10.0,20.0,1.367176e+06,-6.349946e+06,1.367176e+06,17738


In [45]:
results = results.drop(columns=['short_max_k','long_max_k'])

In [16]:
result_df = pd.read_csv('../wlogs/backtests/2023-11-5-ETC-HedgeMode-trend-(sl=0.025).csv')
print(f"max short k={result_df['short_k'].max()}")
print(f"max long k={result_df['long_k'].max()}")

max short k=10
max long k=11


In [18]:
short_k_dist = result_df['short_k'].value_counts()
# print(short_k_dist)
short_k_dist.at[len(short_k_dist)]=0
# print(short_k_dist)
short_k_dist = short_k_dist.diff(-1).dropna()[1:].astype(int)
# print(short_k_dist)
long_k_dist = result_df['long_k'].value_counts()
print(long_k_dist)
long_k_dist.at[len(long_k_dist)]=0
print(long_k_dist)
long_k_dist = short_k_dist.diff(-1).dropna()[1:].astype(int)
print(long_k_dist)


long_k
0     4850
1     2439
2     1222
3      652
4      320
5      176
6       72
7       41
8       13
9        7
11       1
Name: count, dtype: int64
long_k
0     4850
1     2439
2     1222
3      652
4      320
5      176
6       72
7       41
8       13
9        7
11       0
Name: count, dtype: int64
short_k
2    382
3     64
4    134
5     14
6     36
7      1
8     12
9      0
Name: count, dtype: int64
